    SplitNN with two partecipants:

        - Bob has labels.
        - Alice has X values.

     Has two model segments:
        - Alice has the bottom half.
        - Bob has the top half.
     Trains on the MNIST dataset

In [1]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
#hook = sy.TorchHook(torch)

C:\Users\Raffaele\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [31]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)


train_subset = Subset(train_dataset, range(1000))
test_subset = Subset(test_dataset, range(100))

batch_size = 10
train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_subset, batch_size=batch_size, shuffle=False)

In [3]:
for images, labels in train_loader:
    # Check the shape of the batch of images
    print(images.shape)
    break  

#torch.Size([64, 1, 28, 28])  --> torch.Size([batch_size, channels, height, width])  

torch.Size([10, 1, 28, 28])


In [45]:
class Client(nn.Module):
    def __init__(self, x_train, y_train):
        super(Client, self).__init__()

        self.num_samples = x_train.shape[0]
        self.x_train = x_train
        self.labels = y_train
        self.batch_size = 64
        
        self.model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding='valid'),
                                   nn.ReLU())

        self.optimizer = optim.SGD(self.parameters(), lr=1e-2)

    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs

    def send(self, i):
        start, end = i*self.batch_size, (i+1) * self.batch_size
        
        logits = self.forward(self.x_train[start:end])
        labels = self.labels[start:end]
        return logits, labels

    def update(self, g_client, loss):
        self.optimizer.zero_grad()
        g_client = g_client.squeeze(dim=0).clone()
        last = list(self.model.parameters())[-2]
        for param, grad in zip(list(last), g_client):
            if param is not None:
                param -= grad * 1e-2 
        
        torch.autograd.grad(loss, self.model.parameters())

        # Perform the backward pass starting from the gradients received from the server
        self.optimizer.step()

        # Clear the computed gradients
        self.optimizer.zero_grad()

In [5]:
class Server(nn.Module):
    def __init__(self, client):
        super(Server, self).__init__()

        self.num_samples = client.num_samples
        self.client = client
        self.batch_size = client.batch_size
        
        self.model = nn.Sequential(nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding='valid'),
                                   #I need to stop here with grad
                                   
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=(2, 2)),
                                   nn.Flatten(),
                                   nn.Linear(64 * 12 * 12, 10),
                                   nn.Softmax(dim=1))

        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=1e-2)

    def forward(self, split_logits):
        return self.model(split_logits)

    def loss(self, logits, labels):
        return self.loss_function(logits, labels)

    def train(self):
        #g_client_accumulated = []

        for i in range(self.num_samples // self.batch_size):
            self.optimizer.zero_grad()
            self.client.optimizer.zero_grad()

            logits, labels = self.client.send(i)
            final_logits = self.forward(logits)
            loss = self.loss(final_logits, labels)

            # Backward pass for the main model
            g_server = torch.autograd.grad(loss, list(self.parameters())[:-1], retain_graph=True)
            for param, grad in zip(self.parameters(), g_server):
                param.grad = grad
            
            
            
            
            self.optimizer.zero_grad()
            last_layer_params_server = list(self.model.parameters())
            #g_server_last_layer = torch.autograd.grad(loss, last_layer_params_server[0])
            self.optimizer.step()
            # Here I'm sending the gradients of the last layer from the server to the client
            #self.client.update(g_server_last_layer)

            # Backward pass for the client model
            #g_client = torch.autograd.grad(loss, self.client.parameters())
            #for param, grad in zip(self.client.parameters(), g_client):
                #param.grad = grad

            # Apply gradients for both models
            #self.client.optimizer.step()

            #g_client_accumulated.extend(g_client)

        return last_layer_params_server[0], loss
        

    def test(self, inputs, labels):
        split_logits = self.client.forward(inputs)
        logits = self.forward(split_logits)

        _, predictions = torch.max(logits, 1)
        acc = torch.mean((predictions == labels).float())

        return acc.item()

In [46]:
x_train = []
y_train = []
x_test = []
y_test = []

for images, labels in train_loader:
    x_train.append(images)
    y_train.append(labels)
    
for images_test, labels_test in test_loader:
    x_test.append(images_test)
    y_test.append(labels_test)

# Concatenate the batches to obtain the full datasets
x_train = torch.cat(x_train, dim=0)
y_train = torch.cat(y_train, dim=0)

x_test = torch.cat(x_test, dim=0)
y_test = torch.cat(y_test, dim=0)

def main():
    client = Client(x_train, y_train)
    server = Server(client)

    num_rounds = 5
    for i in range(num_rounds):
        
        g_client, loss = server.train()
        #print(g_client[0].shape)
        client.update(g_client, loss)
        
        test_acc = server.test(x_test, y_test)
        print(f"Round {i+1} - TEST ACCURACY: {test_acc * 100:.2f}%")

if __name__ == '__main__':
    main()

RuntimeError: Output 0 of UnbindBackward0 is a view and is being modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.